In [11]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import roc_auc_score

data=pd.read_csv('../Feature Engineering/data_post_pca.csv')

X_pca = data.drop('label', axis=1)
y = data['label']

# from sklearn.naive_bayes import GaussianNB
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)
X_train,X_val,y_train,y_val=train_test_split(X_train, y_train, test_size=0.2, random_state=42)






model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[-1], activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))





Epoch 1/20
783/783 [==============================] - 1s 814us/step - loss: 0.3734 - accuracy: 0.8184 - val_loss: 0.2319 - val_accuracy: 0.8676
Epoch 2/20
783/783 [==============================] - 1s 778us/step - loss: 0.1707 - accuracy: 0.9288 - val_loss: 0.2316 - val_accuracy: 0.8784
Epoch 3/20
783/783 [==============================] - 1s 766us/step - loss: 0.1319 - accuracy: 0.9517 - val_loss: 0.1461 - val_accuracy: 0.9262
Epoch 4/20
783/783 [==============================] - 1s 759us/step - loss: 0.1099 - accuracy: 0.9599 - val_loss: 0.1318 - val_accuracy: 0.9323
Epoch 5/20
783/783 [==============================] - 1s 727us/step - loss: 0.1185 - accuracy: 0.9558 - val_loss: 0.1572 - val_accuracy: 0.9318
Epoch 6/20
783/783 [==============================] - 1s 733us/step - loss: 0.1188 - accuracy: 0.9564 - val_loss: 0.1321 - val_accuracy: 0.9310
Epoch 7/20
783/783 [==============================] - 1s 730us/step - loss: 0.1230 - accuracy: 0.9481 - val_loss: 0.1792 - val_accuracy:

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
y_pred=model.predict(X_test)
y_pred = [1 if x > 0.5 else 0 for x in y_pred]
# y_test contains the actual labels for the test dataset
# y_pred contains the predicted labels by KNN model for the test dataset

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Calculate precision
precision = precision_score(y_test, y_pred)

# Calculate recall
recall = recall_score(y_test, y_pred)

# Calculate F1 score
f1 = f1_score(y_test, y_pred)

# Calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Print the results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:\n", cm)

148/148 [==============================] - 0s 330us/step
Accuracy: 0.9357157961514062
Precision: 0.7342222222222222
Recall: 0.9939831528279182
F1 Score: 0.8445807770961147
Confusion Matrix:
 [[3599  299]
 [   5  826]]


In [9]:
# import numpy as np
# import keras
# from keras.datasets import cifar100
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Flatten
# from keras.layers import Conv2D, MaxPooling2D
# from keras import backend as K
# import matplotlib.pyplot as plt

# def train_model(model):
#   # Train model
#   history = model.fit(x_train, y_train, batch_size=256, epochs=50, validation_data=(x_val, y_val))
#   # Evaluate model on test set
#   score = model.evaluate(x_test, y_test, verbose=0)
#   print('Test accuracy:', score[1])
#   results[score[1]]=model
#   # Plotting the learning curve 
#   train_acc = history.history['accuracy']
#   val_acc = history.history['val_accuracy']

#   epochs = range(1, len(train_acc) + 1)
#   plt.plot(epochs, train_acc, 'bo', label='Training Acc')
#   plt.plot(epochs, val_acc, 'b', label='Validation Acc')
#   plt.title('Training and Validation Acc')
#   plt.xlabel('Epochs')
#   plt.ylabel('Accuracy')
#   plt.legend()
#   plt.show()
#   return val_acc[-1]

In [10]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import roc_auc_score

data = pd.read_csv('../Feature Engineering/data_post_pca.csv')

X_pca = data.drop('label', axis=1)
y = data['label']

# Split the data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define the model
def create_model(loss='binary_crossentropy', activation='relu', optimizer=Adam(lr=0.01), units1=64, units2=128, units3=128, units4=64, units5=32):
    model = Sequential()
    model.add(Dense(units1, input_dim=X_train.shape[-1], activation=activation))
    model.add(Dense(units2, activation=activation))
    model.add(Dense(units3, activation=activation))
    model.add(Dense(units4, activation=activation))
    model.add(Dense(units5, activation=activation))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
    return model

# Define the hyperparameters to tune
param_grid = {
    'loss': ['binary_crossentropy', 'hinge'],
    'activation': ['relu', 'sigmoid'],
    'optimizer': [Adam(lr=0.01)],
    'units1': [32, 64, 128],
    'units2': [64, 128, 256],
    'units3': [64, 128, 256],
    'units4': [32, 64, 128],
    'units5': [16, 32, 64]
}

# Create the model
model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=32, verbose=0)

# Use GridSearchCV to find the best hyperparameters
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# Print the best hyperparameters and the corresponding model accuracy
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Evaluate the model on the test set
_, accuracy = grid_result.best_estimator_.model.evaluate(X_test, y_test)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

# Predict probabilities and calculate AUC
y_prob = grid_result.best_estimator_.model.predict(X_test)
auc = roc_auc_score(y_test, y_prob)
print("AUC: %.2f%%" % (auc * 100.0))


/Users/pratikjadhav/opt/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
/var/folders/g2/vk4vm02n3cng67bqls1m0knw0000gn/T/ipykernel_67353/3159341364.py:46: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=32, verbose=0)
2023-05-03 03:51:29.682705: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-05-03 03:51:29.794913: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-05-03 03:51:29.817014: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-05-03 03:51:29.821931: W tensorflow/core/platform/profi

KeyboardInterrupt: 